## Occasionally dishonest casino
First, we are trying to generate data for the example of the occasionally dishonest casino (see Murphy 17.4). The idea behind the example is that die rolls are generated from a global state, which is either **fair** (each die face is equally likely) or **loaded** (one die face, here 6, has a higher probability than the others). Below the transition diagram is drawn, showing the transition probabilities of the global states along arrows, and individual die roll probabilities of each state inside the box. Let's implement a function that returns rolls as well as the die state for given sequence lengths.

<img src="img/casino.png" width="400">

In [72]:
import numpy as np
np.random.seed(0)

### defintions/parameters
die_states = ['F', 'L']                                    # hidden state (F is fair, L is loaded)
faces = [1, 2, 3, 4, 5, 6]                                 # observable state space
obsModel = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
                    [1/13, 1/13, 1/13, 1/13, 1/13, 8/13]]) # observable state probabilities for given hidden state 
transition_matrix = np.array([[0.99, 0.01],[0.01, 0.99]])   # hidden state transition matrix

### function that returns observed (rolls) and hidden states (die) for given sequence lengths
def rolling_dice(n):
    dice = np.zeros(n, dtype=str)
    rolls = np.zeros(n, dtype=np.int32)
    # Generate random initial hidden state
    start_rv = np.random.rand()
    if start_rv <= 0.5: # initial hidden states are equally likely
        die = 0
    else:
        die = 1
    # Roll the dice for n times
    for i in range(n):
        dice[i] = die_states[die]
        rolls[i] = np.random.choice(faces, p=obsModel[die])
        trans_rv = np.random.rand()
        if trans_rv > transition_matrix[die][die]:
            die ^= 1
    return rolls, dice

### Generating data
Now let's generate data for a sequence length of $T=300$.

In [73]:
from sklearn.preprocessing import OneHotEncoder
T = 10000
rolls, dice = rolling_dice(T)
### this turns the rolls into a six-dimensional binary vector
enc = OneHotEncoder()
rolls_onehot = enc.fit_transform(rolls.reshape(-1, 1)).toarray() 

### Forwards algorithm

In [74]:
from itertools import compress
class HMM:
    def __init__(self, pi_0, A, B, data):
        self.pi = pi_0
        self.A = np.array(A)
        self.B = B
        self.data = data
        self.N = len(self.pi)
        self.T, self.M = self.data.shape
        self.Z = np.zeros(self.T, dtype=np.int)
        self.alpha = np.zeros((self.T, self.N))
        self.beta = np.zeros((self.T, self.N))
        self.delta = np.zeros((self.T, self.N))

    def normalize(self, a):
        return a / np.sum(a)


    def find_B_w_O(self, t):
        result = []
        for i in range(self.N):
            result.append(list(compress(self.B[i], self.data[t]))[0])
        return result

In [75]:
class HMMForwards(HMM):    
    def __init__(self, *args):
        HMM.__init__(self, *args)
    
    def forwards(self):
        # p(q(t)|O(1), ...., O(t))
        unnormalized = np.multiply(self.pi, self.find_B_w_O(0))
        self.alpha[0] = self.normalize(unnormalized)
        self.Z[0] = np.sum(unnormalized)
        logZ = np.log(self.Z[0] + 1e-7)

        for t in range(1, self.T):
            unnormalized = np.multiply(np.dot(self.alpha[t-1], self.A), self.find_B_w_O(t))
            self.alpha[t] = self.normalize(unnormalized)
            self.Z[t] = np.sum(unnormalized)
            logZ += np.log(self.Z[t] + 1e-7)

        return self.alpha, self.Z, logZ

### Define Backwards method